In [ ]:
!pip install openai-whisper
!pip install flask
!pip install python-multipart  # 파일 업로드 시 필요
!pip install torch
!pip install numpy
!pip install librosa
!pip install pandas
!pip install diffusers
!pip install rapidfuzz  # (옵션) 더 빠른 유사도 측정용
!pip install Levenshtein  # (옵션) 정확한 거리 기반 유사도 측정용


In [ ]:
!apt update && apt install -y ffmpeg

In [ ]:
!pip install git+https://github.com/openai/whisper.git
!pip install torch torchaudio
!pip install librosa numpy soundfile

In [ ]:
!pip install hangul-romanize

In [ ]:
!pip install flask-ngrok pyngrok --quiet  # ⬅️ 최초 1회 실행 필요
!pip install pyngrok --quiet

In [ ]:
!ngrok config add-authtoken 2ydg9rJPDW0o04b2Lxoacv8v0pw_6oK7D152GLHfWBuHhpme3

In [ ]:
from flask import Flask, request, jsonify
import whisper
import os
import tempfile
import difflib
import subprocess
import time
import requests

from hangul_romanize import Transliter
from hangul_romanize.rule import academic

# 설정
PORT = 5001
STATIC_DOMAIN = "wise-positively-octopus.ngrok-free.app"

# Whisper 모델
transliter = Transliter(academic)
app = Flask(__name__)
model = whisper.load_model("base")

# 예시 문장
sentence_dict = {
    "1": "오늘은 날씨가 아주 맑아요.",
    "2": "저는 한국어를 배우고 있습니다.",
    "3": "학교에 가는 길에 친구를 만났어요.",
    "4": "사과와 바나나를 샀습니다.",
    "5": "내일 같이 점심 먹을까요?",
    "6": "저녁에 공원에서 산책했어요.",
    "7": "책상 위에 연필이 있어요.",
    "8": "내일은 중요한 시험이 있습니다.",
    "9": "머리가 아파서 병원에 왔어요.",
    "10": "지하철을 타고 회사에 갑니다."
}

def korean_to_roman(text):
    try:
        return transliter.translit(text)
    except Exception as e:
        print("Romanization Error:", e)
        return text.lower()

@app.route('/api/analyze-audio', methods=['POST'])
def analyze_audio():
    try:

        audio_file = request.files['audio']
        sentenceId = request.form['sentenceId']
        userId = request.form.get('userId', 'test-user')

        if sentenceId not in sentence_dict:
            return jsonify({"error": "Invalid sentenceId"}), 400

        with tempfile.NamedTemporaryFile(delete=False, suffix=".wav") as tmp:
            audio_file.save(tmp.name)
            audio_path = tmp.name

        result = model.transcribe(audio_path, language='ko')
        transcript = result["text"].strip()
        os.unlink(audio_path)

        target = sentence_dict[sentenceId]
        user_roman = korean_to_roman(transcript)
        target_roman = korean_to_roman(target)

        matcher = difflib.SequenceMatcher(None, user_roman, target_roman)
        score = round(matcher.ratio() * 100)

        diff_list = []
        for tag, i1, i2, j1, j2 in matcher.get_opcodes():
            if tag == 'equal':
                continue
            error_type = {
                'replace': 'substitute',
                'delete': 'insert',
                'insert': 'delete'
            }.get(tag, 'mismatch')

            diff_list.append({
                "user": user_roman[i1:i2],
                "expected": target_roman[j1:j2],
                "error": error_type
            })

        feedback = "상" if score > 85 else "중" if score > 60 else "하"

        return jsonify({
            "score": score,
            "user_pronunciation": user_roman,
            "target_pronunciation": target_roman,
            "diff": diff_list
        })

    except Exception as e:
        return jsonify({"error": str(e)}), 500

# ngrok 실행
print("Launching ngrok on static domain...")
subprocess.Popen(['ngrok', 'http', f'--domain={STATIC_DOMAIN}', str(PORT)])
time.sleep(5)  # ngrok 연결 대기
print(f"Static domain ready: https://{STATIC_DOMAIN}")

# Flask 실행
app.run(port=PORT)


100%|███████████████████████████████████████| 139M/139M [00:01<00:00, 91.2MiB/s]


Launching ngrok on static domain...
Static domain ready: https://wise-positively-octopus.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5001
INFO:werkzeug:Press CTRL+C to quit
/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
INFO:werkzeug:127.0.0.1 - - [18/Jun/2025 13:51:19] "POST /api/analyze-audio HTTP/1.1" 200 -
/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
INFO:werkzeug:127.0.0.1 - - [18/Jun/2025 13:51:58] "POST /api/analyze-audio HTTP/1.1" 200 -
/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
INFO:werkzeug:127.0.0